### Make plotly plots for measuring features in solar wind data

In [1]:
import pickle
import os
import numpy as np
from sunpy.time import parse_time
import matplotlib.pyplot as plt

data_path='data/'

#Plotly imports
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px
pio.renderers.default = 'browser'

#### print files in data_path

In [2]:
files = os.listdir(data_path)
files.sort()
fnames=[os.path.join(data_path, f) for f in files]
for item in fnames:
    print(item)

data/.DS_Store
data/.ipynb_checkpoints
data/AAREADME_insitu.txt
data/EISN_current.csv
data/HCME_WP3_V06.vot
data/HELIO4CAST_ICMECAT_juno_davies_chris.csv
data/SN_d_tot_V2.0.csv
data/SN_m_tot_V2.0.csv
data/SN_ms_tot_V2.0.csv
data/TableCyclesMiMa.txt
data/bepi_2019_2021_sceq.p
data/bepi_2021_2022_ib_sceq.p
data/davies_2020_paper
data/maven_2014_2018_removed.p
data/maven_2014_2018_removed_smoothed.p
data/maven_2014_2021_removed_no_plasma.p
data/maven_2014_2021_removed_smoothed_no_plasma.p
data/messenger_2007_2015_sceq_removed.p
data/msl_2012_2019_rad.p
data/noaa_rtsw_last100days_now.p
data/observed-solar-cycle-indices_2023_apr_19.json
data/omni2_all_years.dat
data/omni_1963_now.p
data/positions_2007_2023_HEEQ_6hours.sav
data/predicted-solar-cycle-2023_apr_19.json
data/predicted-solar-cycle_2020_may_27.json
data/psp_2018_2022_rtn.p
data/psp_2018_2022_sceq.p
data/psp_2022_add_mag_rtn.p
data/psp_2022_add_mag_sceq.p
data/psp_2022_rtn_new_jan2023.p
data/psp_2022_rtn_sep_oct.p
data/psp_2022_sce

In [3]:
#print('load Solar Orbiter SCEQ')
#filesolo='solo_2020_april_2022_sep_mag_sceq.p'
#solo=pickle.load(open(data_path+filesolo, "rb" ) ) 
#sc=solo
#sc_label='Solar Orbiter'
#start1='2022-Jan-1 00:00'
#end1='2022-Jun-30 00:00'


#print('load Solar Orbiter SCEQ')
#filesolo='solo_2020_april_2022_june_rtn.p'
#solo=pickle.load(open(data_path+filesolo, "rb" ) ) 
#sc=solo
#sc_label='Solar Orbiter'
#start1='2022-Jan-1 00:00'
#end1='2022-Jun-30 00:00'

#print('PSP SCEQ')
#filepsp='psp_2022_add_mag_sceq.p'
#psp=pickle.load(open(data_path+filepsp, "rb" ) ) 
#filepsp='psp_2022_add_mag_rtn.p'
#psp=pickle.load(open(data_path+filepsp, "rb" ) ) 

#sc=psp
#sc_label='PSP'
#start1='2022-Jan-1 00:00'
#end1='2022-Mar-31 00:00'


print('wind HEEQ')
filesolo='wind_2018_now_heeq.p'
[wind,windh]=pickle.load(open(data_path+filesolo, "rb" ) ) 
sc=wind
sc_label='Wind'
#start1='2022-Sep-1 00:00'
#end1='2022-Nov-11 00:00'


#print('load bepi')
#file='bepi_2021_2022_ib_sceq.p'
#file='bepi_2021_2022_ib_sceq.p'

#bepi=pickle.load(open(data_path+file, "rb" ) ) 
#sc=bepi
#sc_label='Bepi'
#start1='2022-Apr-1 00:00'
#end1='2022-Nov-31 00:00'


#print('load sta')
#filesolo='stereoa_2020_now_sceq_beacon.p'
#[sta,stah]=pickle.load(open(data_path+filesolo, "rb" ) ) 
#sc=sta
#sc_label='STEREO-A'
#start1='2022-Feb-1 00:00'
#end1='2022-Nov-11 00:00'



####### select parts for plotting
sc=sc[-100000:-1]

#sc[0:1]



wind HEEQ


### Use plotly for measuring times

#### with plasma data


In [4]:
nrows=4
#init_notebook_mode(connected = True)
#init_notebook_mode(connected = False)

#fig=plt.figure(figsize=(10,6), dpi=150)

fig = make_subplots(rows=nrows, cols=1, shared_xaxes=True)


#for column, color in zip(['b_x', 'b_y', 'b_z', 'b_tot'], ['red', 'green', 'blue', 'black']):
fig.add_trace(go.Scatter(x=sc.time, y=sc.bx, name='Bx',line_color='red'), row=1, col=1)
fig.add_trace(go.Scatter(x=sc.time, y=sc.by, name='By',line_color='green'), row=1, col=1)
fig.add_trace(go.Scatter(x=sc.time, y=sc.bz, name='Bz',line_color='blue'), row=1, col=1)
fig.add_trace(go.Scatter(x=sc.time, y=sc.bt, name='Bt',line_color='black'), row=1, col=1)

fig.add_trace(go.Scatter(x=sc.time, y=sc.vt, name='Vt',line_color='black'), row=2, col=1)
fig.add_trace(go.Scatter(x=sc.time, y=sc.np, name='Np',line_color='black'), row=3, col=1)
fig.add_trace(go.Scatter(x=sc.time, y=sc.tp, name='Tp',line_color='black'), row=4, col=1)

fig.update_layout(title='Wind')

fig.write_html(f'/Users/chris/python/heliocats/measure.html')
fig.show()

### no plasma data

In [7]:
nrows=1
#init_notebook_mode(connected = True)
#init_notebook_mode(connected = False)

#fig=plt.figure(figsize=(10,6), dpi=150)

fig = make_subplots(rows=nrows, cols=1, shared_xaxes=True)

#for column, color in zip(['b_x', 'b_y', 'b_z', 'b_tot'], ['red', 'green', 'blue', 'black']):
fig.add_trace(go.Scatter(x=sc.time, y=sc.bx, name='Bx',line_color='red'), row=1, col=1)
fig.add_trace(go.Scatter(x=sc.time, y=sc.by, name='By',line_color='green'), row=1, col=1)
fig.add_trace(go.Scatter(x=sc.time, y=sc.bz, name='Bz',line_color='blue'), row=1, col=1)
fig.add_trace(go.Scatter(x=sc.time, y=sc.bt, name='Bt',line_color='black'), row=1, col=1)

fig.update_layout(title='Wind')

fig.write_html(f'/Users/chris/python/heliocats/measure.html')
fig.show()